In [1]:
import pandas as pd

news_articles = pd.read_csv('data/news_articles.csv')

In [6]:
topic = news_articles.iloc[0]["content"]

In [40]:
topic

'Follow us today... It is interesting that J.D. Power is giving the 2024 Dodge Daytona Charger EV its J.D. Power 2025 U.S. ALG Residual Value Award in the electric vehicle segment. The Charger EV isn’t even out yet, but the auto experts are expecting great things from the new vehicle. The Charger EV will be Dodge’s first EV. The Jeep Gladiator ranks No. 1 for residual value among midsize pickups. The Jeep Wagoneer earns award for best residual value among large SUVs for third consecutive year. Why J.D. Power is Ranking the Dodge Charger Daytona #1 The J.D. Power annual award projects which vehicle models will hold the highest percentage of their manufacturer’s suggested retail price following a three-year period of ownership. The first edition of any vehicle is often a collector’s edition but J.D Power is banking on the Charger EV holding its value as well as the gas-powered versions do. "The last-generation Dodge Charger won ALG Residual Value Awards for eight consecutive years, from 

In [27]:
# create pydantic class with texts
from pydantic import BaseModel
from typing import List

class Texts(BaseModel):
    text: str

class NewsArticle(BaseModel):
    output: List[Texts]



In [57]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
def generate_voiceover_texts(topic):
    """
    Generate four captivating sentences about a topic using OpenAI's GPT-4.
    
    Args:
        topic (str): The topic to generate sentences about
    """
    client = OpenAI()
    
    prompt = f"""Create 4 captivating, emotionally engaging short sentences about the news below. Requirements:
     - Each sentence MUST be 10-15 words maximum
    - Use active voice and strong verbs
    - Focus on one key point per sentence
    - Avoid complex clauses or conjunctions
    - Include details and numbers where relevant
    - End with a powerful message or call to action
    
    Format the output as 4 separate sentences.
    
    Example flow:
    ["Introduction and hook", "Key benefits or developments", "Practical impact or application", "Future outlook and call to action"]
    
    News:
    {topic}"""

    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert storyteller and marketing copywriter."},
            {"role": "user", "content": prompt}
        ],
        response_format=NewsArticle
    )
    
    # Split the response into individual sentences
    json_str = response.choices[0].message.parsed
    return json_str.output

In [58]:
texts = generate_voiceover_texts(topic)

In [59]:
voiceover_texts = [text.text for text in texts]
voiceover_texts

["Dodge Charger Daytona EV wins J.D. Power's 2025 Residual Value Award for electric cars.",
 'Jeep Gladiator secures top position in midsize pickup residual value ranking.',
 'Jeep Wagoneer claims best large SUV residual value award three years running.',
 'Follow these groundbreaking vehicles reshaping the auto industry; witness the future unfold.']

In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def create_render(texts):
    """
    Create a render using the Creatomate API with a list of voiceover texts.
    
    Args:
        texts (list): List of 4 strings for voiceover texts
    """
    url = "https://api.creatomate.com/v1/renders"
    headers = {
       "Authorization": f"Bearer {os.getenv('CREATOMATE_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    # Define the image URLs
    images = {
        "Image-1": "https://creatomate.com/files/assets/6e448991-32c0-46ff-9043-d0a74fd99b41",
        "Image-2": "https://creatomate.com/files/assets/0f1547eb-647e-429c-a04a-b414d2e2657b",
        "Image-3": "https://creatomate.com/files/assets/08ffb646-af54-4e91-9b38-3eae161fb770",
        "Image-4": "https://creatomate.com/files/assets/e0063a09-ab39-4a41-9c11-359cb339a3db",
    }

    
    # Create modifications dictionary combining images and voiceovers
    modifications = {
        **images,  # Unpack the images dictionary
        **{f"Voiceover-{i+1}": text for i, text in enumerate(texts)}  # Add voiceover texts
    }
    
    payload = {
        "template_id": "16dcc861-b57e-41ac-a83e-ba5623697051",
        "modifications": modifications
    }
    
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

# Make the API call
result = create_render(voiceover_texts)
print(result)

[{'id': '5fe45a4e-651a-4999-bfec-df6991461051', 'status': 'planned', 'url': 'https://cdn.creatomate.com/renders/5fe45a4e-651a-4999-bfec-df6991461051.mp4', 'snapshot_url': 'https://cdn.creatomate.com/snapshots/5fe45a4e-651a-4999-bfec-df6991461051.jpg', 'template_id': '16dcc861-b57e-41ac-a83e-ba5623697051', 'template_name': 'Short-Form Voice Over', 'template_tags': [], 'output_format': 'mp4', 'modifications': {'Image-1': 'https://creatomate.com/files/assets/6e448991-32c0-46ff-9043-d0a74fd99b41', 'Image-2': 'https://creatomate.com/files/assets/0f1547eb-647e-429c-a04a-b414d2e2657b', 'Image-3': 'https://creatomate.com/files/assets/08ffb646-af54-4e91-9b38-3eae161fb770', 'Image-4': 'https://creatomate.com/files/assets/e0063a09-ab39-4a41-9c11-359cb339a3db', 'Voiceover-1': "Dodge Charger Daytona EV wins J.D. Power's 2025 Residual Value Award for electric cars.", 'Voiceover-2': 'Jeep Gladiator secures top position in midsize pickup residual value ranking.', 'Voiceover-3': 'Jeep Wagoneer claims b